In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import shutil
import zipfile
import pickle
import string
import sys

from Bio import SeqIO
from Bio.PDB import MMCIFParser, MMCIFIO
from Bio.PDB import PDBParser, PDBIO, Select
from Bio.PDB import Dice
from Bio.PDB.DSSP import DSSP
from Bio.PDB.SASA import ShrakeRupley
from Bio.PDB import Structure, Model, Chain, Residue, Atom
from DSSPparser import parseDSSP

import concurrent.futures
from multiprocessing import Pool
import subprocess

import Modules.SOLVENTACC as SOLVENTACC
import Modules.CROSSLINK as CROSSLINK
import Modules.DATAGEN as DATAGEN
import Modules.TORSION2 as TORSION2

sys.path.append('/home/bis/230711_JSG/230711_JSG_ProteinComplex/AlphaLink/')
from openfold.data import mmcif_parsing
from openfold.np import protein, residue_constants

import warnings
warnings.filterwarnings("ignore")

2024-09-12 14:35:20.760801: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-12 14:35:20.821081: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-12 14:35:21.769347: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2024-09-12 14:35:23,433] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/bis/anaconda3/lib/python3.10/site-packages/Bio/Data/SCOPData.py:18: BiopythonDeprecationWarning: The 'Bio.Data.SCOPData' module will be deprecated in a future release of Biopython in favor of 'Bio.Data.PDBData.
  warnings.warn(


In [2]:
np.linspace(0,1,11)[:-1]

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# MMCIF to single chain PDB

## For CASP Target

In [ ]:
mmcif_root = "/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/240613_old_pdb/"
mmcifs = os.listdir(mmcif_root)

output_root = '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/240613_old_pdb/'

for mmcif in mmcifs:
    try : 
        mmcif_path = os.path.join(mmcif_root, mmcif)
        DATAGEN.mmcif_to_pdbs(mmcif_path, output_root)
    except : 
        print(mmcif)

## For Training set

In [4]:
mmcif_root = "/data/JSG/pdb_mmcif/"
mmcifs = os.listdir(mmcif_root)

output_root = '/data/JSG/splited_pdb/'
for mmcif in mmcifs:
    try : 
        mmcif_path = os.path.join(mmcif_root, mmcif)
        DATAGEN.mmcif_to_pdbs(mmcif_path, output_root)
    except : 
        print(mmcif)

### For Parerall

In [ ]:
DATAGEN.parallel_processing(mmcifs, mmcif_root, output_root)

Error processing 7anm.cif: Chain id ('aa') exceeds PDB format limit.
Error processing 7all.cif: Chain id ('AAA') exceeds PDB format limit.
Error processing 7dgq.cif: Chain id ('C3') exceeds PDB format limit.
Error processing 7b1y.cif: Chain id ('aa') exceeds PDB format limit.
Error processing 7bay.cif: Chain id ('AA') exceeds PDB format limit.
Error processing 7alx.cif: Chain id ('AAA') exceeds PDB format limit.
Error processing 6cfj.cif: Chain id ('1A') exceeds PDB format limit.
Error processing 7bk5.cif: Chain id ('AAA') exceeds PDB format limit.
Error processing 7bk6.cif: Chain id ('AAA') exceeds PDB format limit.
Error processing 7d5t.cif: Chain id ('3A') exceeds PDB format limit.


# PDB to DSSP

## For Training set

In [ ]:
splited_pdb_root = '/data/JSG/splited_pdb/'
splited_pdbs = os.listdir(splited_pdb_root)

temp = []
for pdb in splited_pdbs:
    temp.append(os.path.join(splited_pdb_root, pdb))
splited_pdbs = temp
splited_dssp_root = '/data/JSG/DSSP_from_splited_pdb/'

for pdb in tqdm(splited_pdbs):
    SOLVENTACC.convert_pdb_to_dssp(pdb, splited_dssp_root)    

# For CASP data

In [ ]:
#"""
#for single process..
pdb_root = './231106_Alphalink_bk/240129_CASP14_15_data/native_pdb/'
pdbs = os.listdir(pdb_root)

dssp_root = './231106_Alphalink_bk/240129_CASP14_15_data/DSSP/'
for pdb in pdbs:
    if pdb[0] == 'T':
        print(pdb)
        pdb_path = os.path.join(pdb_root, pdb)
        SOLVENTACC.convert_pdb_to_dssp(pdb_path, dssp_root)    
#"""

# DSSP to SA

# Traininb data set

In [3]:
DSSP_root = '/data1/JSG/intermediate_data/DSSP_from_splited_pdb/'
DSSPs = os.listdir(DSSP_root)

binned_SA_root = '/data/JSG/SolventAcc_binned/'
binned_SAs = os.listdir(binned_SA_root)

former_SA_root = '/data/JSG/SolventAcc/SolventAcc_from_splited_pdb/'
former_SAs = os.listdir(former_SA_root)

former_feature_root = '/data/JSG/240429_feature_dir_with_binned_SA/'
former_features     = os.listdir(former_feature_root)

after_feature_root = '/data/JSG/240429_feature_dir_with_binned_SA/'
after_features = os.listdir(after_feature_root)

In [4]:
for candidate in tqdm(former_features):
    try :
        feature_path = os.path.join(former_feature_root,candidate)
        code_name, chain = candidate.split('.')[0].split('_')
        protein_name = code_name +'_' + chain
            
        with open(feature_path, 'rb') as file:  # 'rb'는 바이너리 읽기 모드
            feature = pickle.load(file)
        
        solvents = SOLVENTACC.get_binned_solvents_for_input_feature(binned_SA_root, feature, protein_name, save = True)
        
        feature_with_binned_SA_path = os.path.join(after_feature_root, candidate)
        feature_with_binned_SA = feature
        feature_with_binned_SA['solvent_acc_binned'] = solvents['binned']
        feature_with_binned_SA['solvent_acc_real'] = solvents['real']
        feature_with_binned_SA['solvent_acc_mask'] = solvents['mask']
        
        with open(feature_with_binned_SA_path, "wb") as file:
            pickle.dump(feature_with_binned_SA, file)   
            
    except Exception as e:
        print(candidate, e)

  0%|          | 2/16441 [00:03<6:34:27,  1.44s/it]

1c7t_a.pkl Ran out of input


  0%|          | 7/16441 [00:10<6:55:52,  1.52s/it]


KeyboardInterrupt: 

In [5]:
SOLVENTACC.DSSP_to_SA_parerall(former_features, former_feature_root, binned_SA_root, after_feature_root)

1c7n_a.pkl
3c7j_a.pkl1c7t_a.pkl6c72_a.pkl3c7g_a.pkl3c7x_a.pkl3c7t_a.pkl4c76_a.pkl3c72_a.pkl2c7g_a.pkl
1c7k_a.pkl5c74_a.pkl5c7t_a.pkl4c7n_b.pkl



6c75_a.pkl
6c7y_b.pkl
1c7g_a.pkl5k5m_a.pkl
2c7b_a.pkl


6k5g_a.pkl
1k5c_a.pkl4k59_a.pkl5k5d_a.pkl3k57_a.pkl3k5i_a.pkl

3k50_a.pkl5k5i_a.pkl3ic3_a.pkl3ic4_a.pkl5ic7_a.pkl
2ic7_a.pkl4icu_a.pkl2ic6_a.pkl2icj_a.pkl3ic9_a.pkl
5ic1_a.pkl4icm_a.pkl
1icw_a.pkl2ic2_a.pkl
3icr_a.pkl3ic5_a.pkl

3ic1_a.pkl
2icp_a.pkl


2ich_a.pkl



4icw_a.pkl3icu_a.pkl
1icf_a.pkl
2ics_a.pkl2icc_a.pkl4ich_a.pkl3icc_a.pkl4ici_a.pkl4s3j_a.pkl3ic8_a.pkl

1s3j_a.pkl3s3t_a.pkl3s3y_a.pkl4s37_a.pkl
4s38_a.pkl4ic1_a.pkl5icu_a.pkl2ici_a.pkl
2icg_a.pkl3s3l_a.pkl6s3f_a.pkl
4s3o_c.pkl1s35_a.pkl

4s36_a.pkl
2drv_a.pkl
3dra_b.pkl
1drt_a.pkl

3dr2_a.pkl6dr3_a.pkl
2drm_a.pkl2dry_a.pkl6drh_b.pkl2drh_a.pkl5drb_a.pkl3drn_a.pkl3dr3_a.pkl


4drj_a.pkl


6drs_a.pkl
3drf_a.pkl

2dr1_a.pkl2dr3_a.pkl3dr5_a.pkl

6drm_a.pkl


3dra_a.pkl


6dre_a.pkl
3qav_a.pkl


4qas_a.pkl4qag_a.pkl
















  0%|          | 0/16441 [00:00<?, ?it/s]

1c7t_a.pkl Ran out of input
3qay_a.pkl

  0%|          | 2/16441 [00:02<6:36:56,  1.45s/it]


6qai_a.pkl

...

 99%|█████████▉| 16341/16441 [48:31<00:09, 10.18it/s]

1lc5_a.pkl3lcr_a.pkl

3lcc_a.pkl


 99%|█████████▉| 16344/16441 [48:32<00:14,  6.51it/s]

3lcm_a.pkl
3lcz_a.pkl


100%|██████████| 16441/16441 [48:44<00:00,  5.62it/s]


## For CASP targets

In [6]:
#DSSP_root = '/data1/JSG/intermediate_data/DSSP_from_splited_pdb/'
DSSP_root = '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/DSSP/'
DSSPs = os.listdir(DSSP_root)

binned_SA_root = '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/SA_binned/'
binned_SAs = os.listdir(binned_SA_root)

former_feature_root = '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/feature/'
former_features     = os.listdir(former_feature_root)

after_feature_root = '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/feature/'
after_features = os.listdir(after_feature_root)

fasta_root = '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/fasta/'
fastas = os.listdir(fasta_root)

In [7]:
for candidate in tqdm(DSSPs):
    candidate = candidate.split('.dssp')[0] + '.pkl'
    print(candidate)
    try :
        former_feature_path = os.path.join(former_feature_root,candidate)
        with open(former_feature_path, 'rb') as file:  # 'rb'는 바이너리 읽기 모드
            former_feature = pickle.load(file)
        
        #del former_feature['solvent_acc']
        sequence = former_feature['sequence'][0].decode('utf-8')

        residue_length = len(sequence)

        code_name = candidate.split('.')[0]
        protein_name = code_name# +'_' + chain.upper()
        dssp_file_path = os.path.join(DSSP_root,   protein_name) +'.dssp'

        solvents = SOLVENTACC.get_binned_solvents_for_casp_target(binned_SA_root, dssp_file_path, residue_length, sequence, save = True)

        feature_with_binned_SA_path = os.path.join(after_feature_root, candidate)

        feature_with_binned_SA = former_feature
        feature_with_binned_SA['solvent_acc_binned'] = solvents['binned']
        feature_with_binned_SA['solvent_acc_real'] = solvents['real']
        feature_with_binned_SA['solvent_acc_mask'] = solvents['mask']

        with open(feature_with_binned_SA_path, "wb") as file:
            pickle.dump(feature_with_binned_SA, file)   

        #print(binned_solvents[:,0].sum(), binned_solvents[:,10].sum(), binned_solvents.sum(), residue_length)
    except Exception as e:
        print(candidate, e)

  4%|▍         | 2/47 [00:00<00:03, 13.85it/s]

T1042.pkl
T1058.pkl
T1039.pkl
T1050.pkl


 13%|█▎        | 6/47 [00:00<00:03, 10.73it/s]

T1038.pkl
T1025.pkl
T1033.pkl
T1057.pkl
T1065s1.pkl
T1072s2.pkl
T1048.pkl
T1043.pkl


 40%|████      | 19/47 [00:00<00:00, 35.02it/s]

...

T1028.pkl
T1045s2.pkl
T1040.pkl
T1079.pkl
T1046s1.pkl
T1044.pkl


 77%|███████▋  | 36/47 [00:01<00:00, 23.06it/s]

T1078.pkl
T1034.pkl
T1064.pkl
T1026.pkl
T1053.pkl
T1024.pkl
T1080.pkl
T1074.pkl
T1032.pkl
T1076.pkl


100%|██████████| 47/47 [00:02<00:00, 21.41it/s]

T1055.pkl
T1041.pkl
T1137s6.pkl
T1137s6.pkl [Errno 2] No such file or directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/feature/T1137s6.pkl'
T1121.pkl
T1114s1.pkl
T1114s1.pkl [Errno 2] No such file or directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/feature/T1114s1.pkl'
T1137s1.pkl
T1137s2.pkl
T1137s2.pkl [Errno 2] No such file or directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/feature/T1137s2.pkl'


## For Netsurfp 

In [8]:
netsurfp_SA_root = '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/netsurfp_SA/'
netsurfp_SAs = os.listdir(netsurfp_SA_root)

netsurfp_binned_SA_root = '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/netsurfp_SA_binned/'
netsurfp_binned_SAs = os.listdir(netsurfp_binned_SA_root)

former_feature_root = '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/feature/'
former_features     = os.listdir(former_feature_root)

In [9]:
for candidate in tqdm(former_features):
    try :
        former_feature_path = os.path.join(former_feature_root,candidate)
        with open(former_feature_path, 'rb') as file:  # 'rb'는 바이너리 읽기 모드
            former_feature = pickle.load(file)
            
        sequence = former_feature['sequence'][0].decode('utf-8')
        residue_length = len(sequence)

        netsurfp_SA_path = os.path.join(netsurfp_SA_root, candidate)
        
        solvents = SOLVENTACC.get_binned_solvents_for_netsurfp(netsurfp_binned_SA_root, netsurfp_SA_path, residue_length, sequence, save = True)
        
    except Exception as e:
        print(candidate, e)

 15%|█▌        | 8/52 [00:00<00:00, 74.98it/s]

T1090.pkl [Errno 2] No such file or directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/netsurfp_SA/T1090.pkl'
T1027.pkl [Errno 2] No such file or directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/netsurfp_SA/T1027.pkl'
.ipynb_checkpoints.pkl [Errno 21] Is a directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/feature/.ipynb_checkpoints.pkl'


 67%|██████▋   | 35/52 [00:00<00:00, 109.76it/s]

T1036s1.pkl [Errno 2] No such file or directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/netsurfp_SA/T1036s1.pkl'
T1082.pkl [Errno 2] No such file or directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/netsurfp_SA/T1082.pkl'
T1029.pkl [Errno 2] No such file or directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/netsurfp_SA/T1029.pkl'
T1073.pkl [Errno 2] No such file or directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/netsurfp_SA/T1073.pkl'
T1049.pkl [Errno 2] No such file or directory: '/home/bis/230711_JSG/230711_JSG_ProteinComplex/231106_Alphalink_bk/240129_CASP14_15_data/netsurfp_SA/T1049.pkl'


100%|██████████| 52/52 [00:00<00:00, 107.19it/s]


In [11]:
solvents.keys()

dict_keys(['binned', 'real', 'mask'])

In [13]:
solvents['mask']

array([0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.